In [ ]:
import torch
import torch.nn as nn

class MergeLayer(nn.Module):
    """merge input height with image"""
    
    def __init__(self, in_channels, out_channels):
        super(MergeLayer, self).__init__()
        self.h_layer = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(1e-2)
        )
        self.i_layer = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(1e-2)
        )
        
    def forward(self, x_h, x_i):
        output_h = self.h_layer(x_h)
        output_i = self.i_layer(x_i)
        output = torch.cat((output_h, output_i), 1)
        return output
    
ml = MergeLayer(1, 64)
xh = torch.randn(1, 1, 256, 256)
xi = torch.randn(1, 1, 256, 256)
output = ml(xh, xi)
output.shape

In [ ]:
class ConvBlock_(nn.Module):
    """Conv(3x3)-BN-LReLU"""
    
    def __init__(self, in_channels, out_channels, stride=1):
        super(ConvBlock_, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(1e-2)
        )
        
    def forward(self, x):
        output = self.conv(x)
        return output
    
input = torch.randn(1, 64, 256, 256)
conv = ConvBlock_(64, 64, stride=2)
output = conv(input)
output.shape

In [ ]:
class Discriminator(nn.Module):
    """Height Layer + Image Layer-BN"""
    
    def __init__(self):
        super(Discriminator, self).__init__()
        self.merge = MergeLayer(1, 64)
        self.convs = nn.Sequential(
            ConvBlock_(128, 64),
            ConvBlock_(64, 64, 2),
            ConvBlock_(64, 128),
            ConvBlock_(128, 128, 2),
            ConvBlock_(128, 256),
            ConvBlock_(256, 256, 2),
            ConvBlock_(256, 512),
            ConvBlock_(512, 512, 2)
        )
        self.fc1 = nn.Linear(512 * 16 * 16, 200)
        self.lrelu = nn.LeakyReLU(1e-2)
        self.fc2 = nn.Linear(200, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x_h, x_i):
        merged = self.merge(x_h, x_i)
        print(merged.shape)
        output = self.convs(merged)
        
        output = output.view(-1, 512 * 16 * 16)
        print(output.shape)
        vector1 = self.fc1(output)
        relu = self.lrelu(vector1)
        vector2 = self.fc2(relu)
        scalar = self.sigmoid(vector2)
        return scalar
    
input1 = torch.randn(16, 1, 256, 256)
input2 = torch.randn(16, 1, 256, 256)
dis = Discriminator()
output = dis(input1, input2)
output

In [27]:
import torch
import torch.nn.functional as F
import numpy as np
gt = np.random.uniform(0.0, 1.0, (4, 4))
print(gt)
dx, dy = np.gradient(gt)
print(dx)
print(dy)
gt = torch.tensor(gt.copy(), dtype=torch.float32)
gt.unsqueeze_(0)
print(gt)
sobel_x = torch.tensor([[[[0, 0, 0], [-0.5, 0, 0.5], [0, 0, 0]]]], dtype=torch.float32)
print(sobel_x)
grad_gt_x = F.conv2d(gt, sobel_x, stride=1, padding=1)
print(grad_gt_x)

[[0.17855258 0.04707576 0.80932265 0.0092098 ]
 [0.39694728 0.47163023 0.6495401  0.66876157]
 [0.09401776 0.97873243 0.76055761 0.10868372]
 [0.35100421 0.21256217 0.74356919 0.92948081]]
[[ 0.2183947   0.42455447 -0.15978255  0.65955177]
 [-0.04226741  0.46582833 -0.02438252  0.04973696]
 [-0.02297154 -0.12953403  0.04701454  0.13035962]
 [ 0.25698645 -0.76617025 -0.01698841  0.82079709]]
[[-0.13147682  0.31538503 -0.01893298 -0.80011285]
 [ 0.07468295  0.12629641  0.09856567  0.01922147]
 [ 0.88471466  0.33326992 -0.43502435 -0.65187388]
 [-0.13844203  0.19628249  0.35845932  0.18591162]]
tensor([[[0.1786, 0.0471, 0.8093, 0.0092],
         [0.3969, 0.4716, 0.6495, 0.6688],
         [0.0940, 0.9787, 0.7606, 0.1087],
         [0.3510, 0.2126, 0.7436, 0.9295]]])
tensor([[[[ 0.0000,  0.0000,  0.0000],
          [-0.5000,  0.0000,  0.5000],
          [ 0.0000,  0.0000,  0.0000]]]])
tensor([[[ 0.0235,  0.3154, -0.0189, -0.4047],
         [ 0.2358,  0.1263,  0.0986, -0.3248],
         [ 0.

In [18]:
import numpy as np
gt = np.random.uniform(0.0, 1.0, (4, 4))
print(gt)
dx, dy = np.gradient(gt)
print(dx)
print(dy)

[[ 0.0375864   0.0888039  -0.25577957  0.16786481]
 [-0.17382783  0.0219375  -0.01411217  0.00683028]
 [-0.27937991  0.24345719 -0.03643394 -0.16174511]
 [-0.17351774  0.53184328 -0.30042312 -0.16928597]]
[[-0.57638603 -0.06348117  0.08432007 -0.28078356]
 [-0.52516854 -0.21016415  0.12385053  0.14286082]
 [-0.18485537  0.0962345   0.06921285 -0.23889866]
 [ 0.52050565  0.03278181 -0.28135177 -0.10776151]]


In [2]:
def func(a, b, *, c, d):
  print(a, b, c, d)
  
func(1, 2, c=3, d=4)

1 2 3 4


In [ ]:
a = np.random.uniform(low=0.0, high=1.0, size=(4, 4))